In [1]:
## Import necessary libraries

import torch
from diffusers import StableDiffusionPipeline
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import nest_asyncio
import  uvicorn
import base64
from io import BytesIO
from PIL import Image

In [ ]:
## Initialize Stable Diffusion
model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

In [ ]:
## Create FastAPI app and endpoints
app = FastAPI()

class TextPrompt(BaseModel):
    text: str

@app.post("/generate")
async def generate_image(prompt: TextPrompt):
    try:
        # Generate image from text
        image = pipe(prompt.text).images[0]

        # Convert PIL image to base64
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()

        return {"image": img_str}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
## Setup ngrok for public URL
!pip install pyngrok           # type: ignore
from pyngrok import ngrok # type: ignore